In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Parameters

cosPhi=0.95
time=48
m=12
netFactor=0.25
noiseFactor=0.05

<b>Considered network

<!-- ![image.png](https://raw.githubusercontent.com/antoniovitorvb/IST-data-analytics-for-smart-grids/main/project1/DASG%20project1%20topology.png) -->

<img src='https://raw.githubusercontent.com/antoniovitorvb/IST-data-analytics-for-smart-grids/main/project1/DASG%20project1%20topology.png' alt='topology.png' width=400 height='auto'>

<b>Initial Data

In [84]:
#Consumption dataset
s = [[0.0450,    0.0150,    0.0470,    0.0330],
     [0.0250,    0.0150,    0.2480,    0.0330],
     [0.0970,    0.0250,    0.3940,    0.0330],
     [0.0700,    0.0490,    0.0200,    0.4850],
     [0.1250,    0.0460,    0.0160,    0.1430],
     [0.2900,    0.0270,    0.0160,    0.0470],
     [0.2590,    0.0150,    0.0170,    0.0200],
     [0.2590,    0.0160,    0.0280,    0.0160],
     [0.4420,    0.0160,    0.0500,    0.0170],
     [0.2010,    0.0230,    0.0460,    0.0160],
     [0.2060,    0.0490,    0.0220,    0.0240],
     [0.1300,    0.0470,    0.0160,    0.0490],
     [0.0460,    0.0260,    0.0170,    0.0480]]
s = np.array(s)

newConsumers = 5
S = s.copy()
newCol = []
np.random.seed(120)
for i in range(newConsumers):
    flat_S = S.flatten()
    samples = np.random.choice(flat_S, size=m+1, replace=True)
    new_column = samples + np.random.normal(loc=0, scale=noiseFactor**2, size=len(samples))
    new_column = np.round(new_column, 3)

    S = np.c_[S, new_column]
    # S.append()

#topology
topo=[[1, 2],[2,3], [2,5], [3,4], [3,5], [5,6]]
nBUS=np.max(topo)

#Impedance
z = []
for i in range(len(topo)):
    r = np.round(np.random.uniform(low=0.01, high=0.25), 3)
    im = np.round(np.random.uniform(low=0.01, high=0.15), 3)

    z.append(complex(r, im))

z = np.multiply(z, netFactor)
# print(z)

vr=1 #Reference voltage
el=1
ni=20 #Iterations for the Power Flow

print('\nApparent power with new consumers:\n', S)

[0.0575 +0.0175j  0.03725+0.01475j 0.056  +0.02375j 0.03925+0.01925j
 0.02525+0.0105j  0.05975+0.0255j ]

Apparent power with new consumers:
 [[0.045 0.015 0.047 0.033 0.013 0.025 0.028 0.024 0.261]
 [0.025 0.015 0.248 0.033 0.046 0.015 0.017 0.026 0.395]
 [0.097 0.025 0.394 0.033 0.028 0.02  0.012 0.014 0.02 ]
 [0.07  0.049 0.02  0.485 0.016 0.022 0.014 0.203 0.024]
 [0.125 0.046 0.016 0.143 0.397 0.071 0.028 0.052 0.047]
 [0.29  0.027 0.016 0.047 0.051 0.203 0.048 0.017 0.034]
 [0.259 0.015 0.017 0.02  0.291 0.043 0.046 0.045 0.01 ]
 [0.259 0.016 0.028 0.016 0.026 0.024 0.203 0.099 0.051]
 [0.442 0.016 0.05  0.017 0.128 0.027 0.396 0.046 0.053]
 [0.201 0.023 0.046 0.016 0.047 0.147 0.045 0.017 0.051]
 [0.206 0.049 0.022 0.024 0.015 0.025 0.053 0.019 0.05 ]
 [0.13  0.047 0.016 0.049 0.142 0.017 0.047 0.267 0.025]
 [0.046 0.026 0.017 0.048 0.267 0.014 0.131 0.017 0.016]]


<b>Power Flow Function

In [136]:
def pf3ph(t,z,si,vr,el,ni,al):
    #Matrices creation
    t=np.array(t)
    p=t[:,0]
    f=t[:,1]
    
    w=len(p)+1
    vp=np.zeros((nBUS-1,w), dtype=complex)
    vn=np.zeros((nBUS-1,w), dtype=complex)
    vp[0,0:w]=vr
    
    for h in range (2,nBUS):
        vp[h-1,:]=vp[h-2,:]*al  #Create a three phase system of voltages
                                #Voltages will be the same in all BUS
        # print(vp.shape, vp)

    va=vp-vn                                                      #Auxiliar voltage
    ia=np.conj(np.divide(np.multiply(si,np.abs(va)**el),va))      #Auxiliar current 
    
    for it in range(ni):                                          #Iterations of Power Flow
        va=vp-vn
        ip=np.conj(np.divide(np.multiply(si,np.abs(va)**el),va))  #Phase current 
        inn=-np.sum(ip,0)                                         #Neutral current 
        for k in range(w-1,0,-1):                                 #Backward Cycle
            n=f[k-1]
            m=p[k-1]
            ip[:,m-1]=ip[:,m-1]+ip[:,n-1]                         #Phase Current
            inn=-np.sum(ip,0)                                     #Neutral Current

        eps= np.linalg.norm(np.max(np.abs(ia-ip),0))              #Error, comparing the new currents and the old ones (previous iteration)

        if eps>1e-4:
            ia=ip
            mvp=0
            mvn=0
            eps=np.inf
        else:                       #If the error is lower than the limit, we can return the results 
            mvp=(vp-vn)             #Phase Voltages to return
            mvn=vn[0,:]             #Neutral Voltage to return
#            return mvp, mvn, eps, ip, inn;
            return mvp
            # return mvp, ip, inn
            
        for k in range (w-1):                     #Forward Cycle
            n=f[k]                                
            m=p[k]
            vn[:,n-1]=vn[:,m-1]-z[k]*inn[n-1]     #Neutral Voltage 
            vp[:,n-1]=vp[:,m-1]-z[k]*ip[:,n-1]    #Phase Voltage
        ia=ip             #Save the current of previous iteration

In [132]:
seed = 0
while True:
    np.random.seed(seed)

    consumerPhase = np.array(np.random.randint(0, 3, size=S.shape[1]))
    consumerBus = np.array(np.random.randint(0, nBUS, size=S.shape[1]))
    # print(np.unique(consumerPhase, return_counts=True))
    # print(np.unique(consumerBus))
    # print(consumerPhase)
    # print(consumerBus)

    # Phase-Bus coordinates
    coordinates = np.vstack((consumerPhase, consumerBus)).T
    unique_coordinates, indices, counts = np.unique(coordinates, axis=0, return_index=True, return_counts=True)

    # Check if any coordinate pairs are repeated
    is_repeated = len(coordinates) == len(unique_coordinates)
    all_phases = np.min(np.unique(consumerPhase, return_counts=True)[1]) >= 2
    all_buses = len(np.unique(consumerBus)) == 6
    if is_repeated and all_phases and all_buses:
        print('right seed =', seed)
        print(consumerPhase)
        print(consumerBus)
        break
    else:
        seed+=1

right seed = 9
[2 0 2 1 2 0 2 1 1]
[4 3 0 2 1 0 5 3 1]


In [135]:
#Creation of Matrices
al = np.exp(-1j * np.deg2rad(120)) # Phase Angle
Y=np.zeros((3*m), dtype=complex)
X=np.zeros((3*m,m), dtype=complex)
v=np.zeros((m,3))
dv_abs=np.zeros((m,3))

# Assign each consumer to one Phase and one Bus:
np.random.seed(seed)
consumerPhase = np.array(np.random.randint(0, 3, size=S.shape[1]))  # 0 = a; 1 = b; 2 = c
consumerBus = np.array(np.random.randint(0, nBUS, size=S.shape[1])) # except the source (n)

# print(consumerPhase)
# print(consumerBus)

si = np.zeros((3, nBUS))
for i in range(m):
    for c in range(S.shape[1]):
        si[consumerPhase[c], consumerBus[c]] = S[i, c]
    # print('\n',S[i,:])
    # print('SI values:\n', pd.DataFrame(si))
    
    # mvp, Ip, Inn = pf3ph(topo,z,si,vr,el,ni,al)
    mvp = pf3ph(topo,z,si,vr,el,ni,al)
    # noise=1+noiseFactor*np.random.randn(3)
    # mvp[:,3]=np.multiply(mvp[:,3],noise)                       #Add noise to the voltages
    # Y[3*(i):3*(i)+3]=mvp[:,3]                                  #Save the voltages in matrix Y
    # dv_abs[i,:]=vr-np.abs(mvp[:,3])                            #Volage variations (only to plot)

Volt=np.reshape(Y,(m,3))   

# print ('The voltages measured in the PMUs are:\n',Volt)
# print ('\nThe Phase Currents measured in the PMUs are:\n', Ip)
# print ('\nThe Neutral Currents measured in the PMUs are:\n', Inn)

#The value of Z is the multiplication between D and W but not considering the values of z_nm; z_ml and z_lk
Z = [[2, al, al**2],
     [1, 2*al, al**2],
     [1, al, 2*al**2]]

[2 0 2 1 2 0 2 1 1]
[4 3 0 2 1 0 5 3 1]
(5, 7) [[ 1. +0.j         1. +0.j         1. +0.j         1. +0.j
   1. +0.j         1. +0.j         1. +0.j       ]
 [-0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j
  -0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j]
 [ 0. +0.j         0. +0.j         0. +0.j         0. +0.j
   0. +0.j         0. +0.j         0. +0.j       ]
 [ 0. +0.j         0. +0.j         0. +0.j         0. +0.j
   0. +0.j         0. +0.j         0. +0.j       ]
 [ 0. +0.j         0. +0.j         0. +0.j         0. +0.j
   0. +0.j         0. +0.j         0. +0.j       ]]
(5, 7) [[ 1. +0.j         1. +0.j         1. +0.j         1. +0.j
   1. +0.j         1. +0.j         1. +0.j       ]
 [-0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j
  -0.5-0.8660254j -0.5-0.8660254j -0.5-0.8660254j]
 [-0.5+0.8660254j -0.5+0.8660254j -0.5+0.8660254j -0.5+0.8660254j
  -0.5+0.8660254j -0.5+0.8660254j -0.5+0.8660254j]
 [ 0. +0.j         0. +0.j         0. +0.j  

ValueError: operands could not be broadcast together with shapes (3,6) (5,7) 